# 각 부분에 대한 좌표값 및 포인트 번호 확인

<img src="https://i.imgur.com/3j8BPdc.png" style="height:400px">

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
## 33개 확인

for mark in mp_pose.PoseLandmark:
    print(mark)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


# 각도 계산

In [4]:
def calculate_angle(a,b,c):
    
    a = np.array(a)  # 첫 지점
    b = np.array(b)  # 중간 지점
    c = np.array(c)  # 끝 지점
    
    ## arctan2의 출력 범위는 [-pi, pi]
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

In [5]:
## 스쿼트 운동

def Squat():

    Squat_left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

    Squat_left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

    Squat_left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]


    Squat_right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

    Squat_right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

    Squat_right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
    
    
    Squat_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

    Squat_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
               landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

    Squat_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    
    return Squat_left_hip, Squat_left_knee, Squat_left_ankle, Squat_right_hip, Squat_right_knee, Squat_right_ankle, Squat_shoulder, Squat_hip, Squat_knee

In [6]:
cap = cv2.VideoCapture(0)

# Curl counter variable
Count = 0
Posture = None
Set = 0
Type = None

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        ## opencv는 bgr 형태이기 때문에 영상 컬러를 조정 해야 함
        video = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # 웹캠에서 bgr에서 rgb로 변경
        video.flags.writeable = False                     # false로 쓰기 불가능한 상태로 설정
        
        # Make detection
        results = pose.process(video)                     # 감지

        video.flags.writeable = True                      # True로 쓰기 가능한 상태로 설정
        video = cv2.cvtColor(video, cv2.COLOR_RGB2BGR)    # rgb에서 bgr로 변경하여 opencv로 작동하도록 설정
        
        
        try:
            # 33개의 좌표값 추출
            landmarks = results.pose_landmarks.landmark
            
            ## 스쿼트 운동 좌표 변수 생성
            Squat_left_hip, Squat_left_knee, Squat_left_ankle, Squat_right_hip, Squat_right_knee, Squat_right_ankle, Squat_shoulder, Squat_hip, Squat_knee = Squat()
            
            
            ## 각도 계산 (스쿼트)
            Squat_angle_left = round(calculate_angle(Squat_left_hip, 
                                                      Squat_left_knee, Squat_left_ankle), 2)
            Squat_angle_right = round(calculate_angle(Squat_right_hip, Squat_right_knee, 
                                                        Squat_right_ankle), 2)
            Squat_angle_waist = round(calculate_angle(Squat_shoulder, Squat_hip, 
                                                        Squat_knee), 2)            
            
            ## 웹캠으로 시각화
            
            ### 스쿼드
            cv2.putText(video, str(Squat_angle_left), 
                        tuple(np.multiply(Squat_left_knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(video, str(Squat_angle_right), 
                        tuple(np.multiply(Squat_right_knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(video, str(Squat_angle_waist), 
                        tuple(np.multiply(Squat_hip, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)   
            
            ## 횟수 검출 
            ### 스쿼트
            if Squat_angle_left > 170 and Squat_angle_right > 170 and Squat_angle_waist > 170:
                Posture = "stand"

                    
            elif Squat_angle_left < 160 or Squat_angle_right < 160:
                if Squat_angle_waist > 155 and Posture == "stand":
                    Posture = "squat"
                    Type = "squat"
                    Count += 1
                    
                    if Count == 10:
                        Set += 1
                        Count = 0
                        print(Set+"세트", Count+"개") 

                
        except:
            pass
        
        ## 1. 횟수 부분을 영상에 표시
        cv2.rectangle(video, (0, 0), (280, 73), (0, 128, 0), -1)
        
        ### 1.1 Type
        cv2.putText(video, "TYPE", (25, 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(video, Type, (10, 65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)        
        
        ### 1.2 SET
        cv2.putText(video, "SET", (120, 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(video, str(Set), (125, 65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        ### 1.3 COUNT
        cv2.putText(video, "COUNT", (200, 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(video, str(Count), (220, 65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    
        ### 1.4 posture
        cv2.putText(video, "Posture", (300, 15), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(video, Posture, (300, 65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
        
        ## 2. 각 관절 포인트 부분을 영상에 표시
        mp_drawing.draw_landmarks(video, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        
        cv2.imshow("Measuring Exercise", video)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()